In [1]:
import numpy as np
from pathlib import Path
from allensdk.core.reference_space_cache import ReferenceSpaceCache

cache_dir = Path("../reference_space_cache")

# Parameters
VOXEL = 25   # µm (use 10, 25, 50...)

# Load CCF annotation and structure tree
# Manifest will be cached locally in this directory
rsc = ReferenceSpaceCache(resolution=VOXEL,
                          reference_space_key="annotation/ccf_2017",
                          manifest=str(cache_dir / "manifest.json"))

annot, meta = rsc.get_annotation_volume()
tree = rsc.get_structure_tree()

In [2]:
# Note: The axes order in the meta data 'space' was wrong.
print(annot.shape)
dict(meta)

(528, 320, 456)


{'type': 'unsigned int',
 'dimension': 3,
 'space': 'left-posterior-superior',
 'sizes': array([528, 320, 456]),
 'space directions': array([[25.,  0.,  0.],
        [ 0., 25.,  0.],
        [ 0.,  0., 25.]]),
 'kinds': ['domain', 'domain', 'domain'],
 'endian': 'little',
 'encoding': 'gzip',
 'space origin': array([0., 0., 0.])}

In [3]:
# Corrected CCF axis order should be: (AP, DV, ML) (per Allen docs)
# Directions: posterior, inferior, right
AP_AXIS, DV_AXIS, ML_AXIS = 0, 1, 2

In [4]:
# Get structure for VISp
ecephys_structure = tree.get_structures_by_acronym(['VISp'])[0]
ecephys_structure

{'acronym': 'VISp',
 'graph_id': 1,
 'graph_order': 185,
 'id': 385,
 'name': 'Primary visual area',
 'structure_id_path': [997, 8, 567, 688, 695, 315, 669, 385],
 'structure_set_ids': [396673091,
  112905828,
  688152357,
  691663206,
  687527945,
  12,
  184527634,
  514166994,
  112905813,
  167587189,
  114512891,
  114512892],
 'rgb_triplet': [8, 133, 140]}

In [5]:
# Get structure for VISp layers
layer_structures = tree.children([ecephys_structure['id']])[0]
layer_structures = {s['acronym']: s for s in layer_structures}

In [6]:
# Get hemisphere annotation
ml_size = annot.shape[ML_AXIS]
ml_mid = ml_size // 2  # midline index

hemisphere = 'left'  # 'left' or 'right'

ml_slice = [slice(None)] * 3
ml_slice[ML_AXIS] = slice(0, ml_mid) if hemisphere == 'left' else slice(ml_mid, ml_size)
hemi_annot = annot[tuple(ml_slice)]

In [7]:
# Loop over layers, compute DV range
ranges = {}
for acr, struct in layer_structures.items():
    coords = np.argwhere(hemi_annot == struct['id'])  # (AP, DV, ML)
    if coords.size == 0:
        continue

    ranges[acr]  = VOXEL * np.array([
        [coords[:,AP_AXIS].min(), coords[:,AP_AXIS].max()],
        [coords[:,DV_AXIS].min(), coords[:,DV_AXIS].max()],
        [coords[:,ML_AXIS].min(), coords[:,ML_AXIS].max()]
    ])


# Print results
print(f"{hemisphere.capitalize()} hemisphere (AP, DV, ML in rows; [min, max] in µm):\n")
for acr, mat in ranges.items():
    print(f"{acr}:\n{mat}\n")

Left hemisphere (AP, DV, ML in rows; [min, max] in µm):

VISp1:
[[ 7675 10350]
 [  375  1950]
 [ 2050  4350]]

VISp2/3:
[[ 7700 10250]
 [  475  1950]
 [ 2125  4350]]

VISp4:
[[ 7700 10100]
 [  675  2000]
 [ 2250  4300]]

VISp5:
[[ 7725 10050]
 [  725  2150]
 [ 2325  4250]]

VISp6a:
[[7725 9875]
 [ 975 2275]
 [2475 4150]]

VISp6b:
[[7750 9600]
 [1125 2300]
 [2550 4100]]



In [ ]:
import matplotlib.pyplot as plt

import add_path
from toolkit.plots.utils import set_equal_3d_scaling

%matplotlib qt


scatter_density = 5000.  # voxels per mm^3

# create colors dictionary using colormap
cmap = plt.get_cmap('rainbow_r', len(layer_structures))
colors = {acr: cmap(i) for i, acr in enumerate(layer_structures)}

_, ax = plt.subplots(1, 1, figsize=(10, 8), subplot_kw=dict(projection='3d'))

for acr, struct in layer_structures.items():
    mask = (hemi_annot == struct['id'])
    coords = np.argwhere(mask)
    if coords.size == 0:
        continue

    # Convert voxel indices to microns
    AP = coords[:, AP_AXIS] * VOXEL
    DV = coords[:, DV_AXIS] * VOXEL
    ML = coords[:, ML_AXIS] * VOXEL
    # Plot only a subsample for speed
    n_voxels = coords.shape[0]
    n_samples = min(int(n_voxels * (VOXEL / 1000.) ** 3 * scatter_density), n_voxels)
    idx = np.random.choice(n_voxels, size=n_samples, replace=False)
    ax.plot(ML[idx], AP[idx], DV[idx], '.', color=colors[acr], markersize=2, label=acr, alpha=0.5)
    # ax.plot(ML, AP, DV, '.', color=colors[acr], markersize=2, label=acr, alpha=0.5)

ax.set_xlabel('ML (µm)')
ax.set_ylabel('AP (µm)')
ax.set_zlabel('DV (µm)')
ax.legend()
ax.set_title(f'{hemisphere.capitalize()} VISp Cortex')

data_lim = set_equal_3d_scaling(ax)

ax.set_ylim3d(data_lim[1][::-1])  # flip AP
ax.set_zlim3d(data_lim[2][::-1])  # flip DV

plt.show()


In [9]:
import pandas as pd

channels_coords = pd.read_csv('../temp/channels_coords.csv', index_col='id')

In [10]:
def world_um_to_idx(x):
    """
    Convert world coordinates (µm) to nearest voxel indices (AP, DV, ML).
    Clips to valid bounds.
    """
    idx = np.round(np.asarray(x) / VOXEL).astype(int)
    idx = np.clip(idx, 0, np.array(annot.shape) - 1)
    return idx

In [11]:
channels_coords.shape

(24, 3)

In [12]:
ccf_idx = world_um_to_idx(channels_coords.values).T
structure_array = tree.get_structures_by_id(annot[tuple(ccf_idx)])
structure_acronym = [s['acronym'] if s else '' for s in structure_array]

In [13]:
structure_acronym

['or',
 'VISp6b',
 'VISp6a',
 'VISp6a',
 'VISp6a',
 'VISp6a',
 'VISp5',
 'VISp5',
 'VISp5',
 'VISp5',
 'VISp5',
 'VISp4',
 'VISp4',
 'VISp4',
 'VISp2/3',
 'VISp2/3',
 'VISp2/3',
 'VISp2/3',
 'VISp2/3',
 'VISp2/3',
 'VISp1',
 'VISp1',
 'VISp1',
 '']